In [1]:
from fastai.text import *
import json
import html
import re
import pickle
from collections import Counter
import random
import pandas as pd
import numpy as np
from pathlib import Path
import sklearn
from sklearn import model_selection
from functools import partial
from collections import Counter, defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.utils 
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import dataset, dataloader
import torch.optim as optim
import torch.nn.functional as F

import time
import math
import sys
import data

snli_root = './data/SNLI/'

In [2]:
# load and process the all the sentences, just to get the LM trained
raw_text = []
for file in [f"{snli_root}/snli_dev.json", f"{snli_root}/snli_test.json"]:
    with open(file) as fp:
        content = json.load(fp)
        for item in content:
            raw_text.append(item[0])
            raw_text.append(item[1])
print(len(raw_text))

39332


In [3]:
#split the language model data into train and validation sets
lm_train, lm_valid = sklearn.model_selection.train_test_split(raw_text, test_size=0.1)
df_trn = pd.DataFrame(lm_train)
df_val = pd.DataFrame(lm_valid)

In [4]:
BOS = 'x_bos'  # beginning-of-sentence tag

re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

def get_texts(df):
    texts = df[0].astype(str)
    texts = list(texts.apply(fixup).values)
    texts = f'{BOS} ' + df[0].astype(str)
    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok

In [5]:
tok_trn = np.concatenate(get_texts(df_trn))
tok_val = np.concatenate(get_texts(df_val))

In [6]:
tok_val[:100]

array(['x_bos', 'a', 'man', 'with', 'an', 'orange', 'jacket', ',', 'takes', 'the', 'photo', 'of', 'four',
       'women', ',', 'and', 'a', 'man', 'in', 'a', 'greenish', 'yellow', 'suite', '.', 'x_bos', 'a', 'young',
       'man', 'attempts', 'to', 'pole', 'vault', 'over', 'a', 'bar', ',', 'barely', 'inches', 'away', 'from',
       'it', 'mid', 'flight', '.', 'x_bos', 'one', 'woman', 'really', 'sucks', 'at', 'playing', 'ping',
       'pong', '.', 'x_bos', 'in', 'a', 'little', 'league', 'baseball', 'game', ',', 'the', 'batter',
       'watches', 'as', 'the', 'ball', 'he', 'has', 'just', 'hit', 'flies', 'off', 'toward', 'right',
       'field', ',', 'with', 'the', 'catcher', 'looking', 'on', '.', 'x_bos', 'two', 'women', 'are', 'in',
       'a', 'car', 'with', 'a', 'man', '.', 'x_bos', 'the', 'ball', 'is', 'on'], dtype='<U14')

In [7]:
#save our work
np.save(f'{snli_root}tok_trn.npy', tok_trn)
np.save(f'{snli_root}tok_val.npy', tok_val)

In [20]:
tok_trn = np.load(f'{snli_root}tok_trn.npy')
tok_val = np.load(f'{snli_root}tok_val.npy')

In [21]:
freq = Counter(np.concatenate([tok_trn, tok_val]))
freq.most_common(25)

[('a', 54168),
 ('x_bos', 39332),
 ('.', 34564),
 ('the', 19411),
 ('in', 15767),
 ('is', 13604),
 ('man', 9985),
 ('on', 8761),
 ('and', 8388),
 ('of', 7605),
 ('are', 7312),
 ('with', 6549),
 (',', 5304),
 ('woman', 5225),
 ('to', 4493),
 ('two', 4440),
 ('people', 4242),
 ('at', 3603),
 ('wearing', 3212),
 ('an', 3059),
 ('his', 2943),
 ('shirt', 2606),
 ('young', 2483),
 ('while', 2405),
 ('men', 2287)]

In [22]:
len(freq)

8840

In [23]:
max_vocab = 60000
min_freq = 1
itos = [o for o, c in freq.most_common(max_vocab) if c>=min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')
stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

In [24]:
len(stoi)

8842

In [25]:
trn_lm = np.array([stoi[p] for p in tok_trn])
val_lm = np.array([stoi[p] for p in tok_val])

In [26]:
#save results
pickle.dump(itos, open(f'{snli_root}itos.pkl', 'wb'))
np.save(f'{snli_root}trn_lm.npy', trn_lm)
np.save(f'{snli_root}val_lm.npy', val_lm)

In [27]:
#load the results so we can pick it up from here 
itos = pickle.load(open(f'{snli_root}itos.pkl', 'rb'))
trn_lm = np.load(f'{snli_root}trn_lm.npy')
val_lm = np.load(f'{snli_root}val_lm.npy')

stoi = defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
vocab_size = len(itos)
vocab_size

8842

In [28]:
for word in val_lm[:100]:
    print(itos[word], end=" ")

x_bos a man with an orange jacket , takes the photo of four women , and a man in a greenish yellow suite . x_bos a young man attempts to pole vault over a bar , barely inches away from it mid flight . x_bos one woman really sucks at playing ping pong . x_bos in a little league baseball game , the batter watches as the ball he has just hit flies off toward right field , with the catcher looking on . x_bos two women are in a car with a man . x_bos the ball is on 